<a href="https://colab.research.google.com/github/Sahithi246/SER/blob/main/SER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install librosa soundfile numpy sklearn pyaudio

  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-aq5j7y9e/pyaudio_71b642e04d7443c69ed7f631531e44c0/setup.py'"'"'; __file__='"'"'/tmp/pip-install-aq5j7y9e/pyaudio_71b642e04d7443c69ed7f631531e44c0/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-b2i44y6t/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pyaudio Check the logs for full command output.


Step-1: Import necessary libraries


In [8]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Step-2: Understanding RAVDEES dataset
RAVDESS stands for Ryerson Audio-Visual Database of Emotional Speech and Song. It is a large dataset will an audio and video database. The original size of this data is around 24Gb. But we will use a smaller portion of it and not the whole dataset. And I downloades it from kaggle. This portion of the RAVDESS contains 1440 files: 60 trials per actor x 24 actors = 1440. The data contains 24 professional actors: 12 female and 12 male. Speech emotions include calm, happy, sad, angry, fearful, surprise, and disgust expressions.
EMOTION LABELS: we are creating a list of emotions that we want to focus in this project. It’s hard to do a prediction using all emotions, because the speech may sound in more than one emotion simultaneously, and that will affect our prediction scores. That’s why I chose three primary emotions, which are happy, sad,angry and fearful.

In [9]:
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions = ['happy', 'sad', 'angry']

Step-3: Extracting features from Audio Recordings:
In this step, I defined a function which will extract audio recordings and return them as stack arrays in sequence horizontally using a numpy hstack method.

In [18]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

Step-4 : Loading and preparing the data

In [14]:
import os
audio_dir = '../content/drive/My Drive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav'
train_dir = os.path.join(audio_dir)

In [15]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(train_dir):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [20]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [21]:
print((x_train.shape[0], x_test.shape[0]))

(432, 144)


In [22]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [23]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,),
                    learning_rate='adaptive', max_iter=500)

In [24]:
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [25]:
y_pred=model.predict(x_test)

In [26]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 78.47%
